In [ ]:
import pandas as pd
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt
# from plotly.subplots import make_subplots
import xarray as xr
import datetime
import os,ast
# import plotly.graph_objects as go
# from sklearn import linear_model
import statsmodels.api as sm
import glob
import warnings
from sklearn import linear_model
# import statsmodels.api as sm
# import ee
# import geopandas as gpd
import pickle
# from scipy.optimize import curve_fit
# from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings("ignore") 
import itertools
from joblib import Parallel, delayed
from multiprocessing import Pool
import time

import dask.dataframe as dd
from dask.multiprocessing import get
import ast
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
### The following script executes the Hybrid Curve fitting approach and the Monte Carlo simulations

In [ ]:
### Defining the Hybrid Curve Function. Hybrid curve is the hybrid of Robust curve and 
### Hydrological parameter derived curve.
def compute(subset,location):
        subset_dataset = f[f['Name'].isin(list(subset))].reset_index(drop=True)

        x = subset_dataset[['constant','Area','temperature','precipitation','TPI']]
        x=sm.add_constant(x)
        y = subset_dataset['esm_mean (millionm3)']
    #     x = sm.add_constant(x) # adding a constant
    #     print('x',x)
        model = sm.OLS(y, x).fit()
    #     print('Params: ',model.params)
    #     print(model.summary())
        ml_area_coef = model.params[1]
        ml_temp_coef = model.params[2]
        ml_ppt_coef = model.params[3]
        ml_tpi_coef = model.params[4]
        ml_constant_coef = model.params[0]
        subset_dataset['CumulativeMaxVolume'] = subset_dataset['Mean_Vol(km3)'].cummax()
        subset_dataset = subset_dataset[subset_dataset['Mean_Vol(km3)'] >= subset_dataset['CumulativeMaxVolume']]
        subset_dataset = subset_dataset.reset_index(drop=True)
        slope, _, r_value, _, _ = linregress(subset_dataset['Area'], subset_dataset['CumulativeMaxVolume'])
        if location =='NW':
            
            stats_area['Hybrid_Vol'] = np.where(
                area_greater_than_30,
                slope * stats_area['Area (inkm2, value*10*10/(1000*1000))'],
                (ml_area_coef * stats_area['Area (inkm2, value*10*10/(1000*1000))'] +ml_temp_coef * (30) + ml_ppt_coef * (350) + (ml_tpi_coef) * (0.39) + ml_constant_coef) / 1000
            )
        elif location == 'SE':
            stats_area['Hybrid_Vol'] = np.where(
                area_greater_than_30,
                slope * stats_area['Area (inkm2, value*10*10/(1000*1000))'],
                (ml_area_coef * stats_area['Area (inkm2, value*10*10/(1000*1000))'] +ml_temp_coef * (28.5) + ml_ppt_coef * (500) + (ml_tpi_coef) * (0.4) + ml_constant_coef) / 1000
            )
        elif location == 'SW':
            stats_area['Hybrid_Vol'] = np.where(
                area_greater_than_30,
                slope * stats_area['Area (inkm2, value*10*10/(1000*1000))'],
                (ml_area_coef * stats_area['Area (inkm2, value*10*10/(1000*1000))'] +ml_temp_coef * (30) + ml_ppt_coef * (340) + (ml_tpi_coef) * (0.33) + ml_constant_coef) / 1000
            )
        elif location =='NE':
            stats_area['Hybrid_Vol'] = np.where(
                area_greater_than_30,
                slope * stats_area['Area (inkm2, value*10*10/(1000*1000))'],
                (ml_area_coef * stats_area['Area (inkm2, value*10*10/(1000*1000))'] +ml_temp_coef * (32) + ml_ppt_coef * (450) + (ml_tpi_coef) * (0.32) + ml_constant_coef) / 1000
            )
        stats_area['Hybrid_Vol']  = stats_area['Hybrid_Vol'].apply(lambda x: x if x >= 0 else 0)
        cumm_vol = stats_area['Hybrid_Vol'].sum()
        return (subset, slope, cumm_vol)


In [ ]:
regions = ['NE','NW','SE','SW']
# f_files is the collection of A-V data points from the different regimes
f_files = ['Complete_waterdata_NE.csv','Complete_waterdata_NW.csv','Complete_waterdata_SE.csv','Complete_waterdata_SW.csv']
# area_files is the area of the waterbodies extracted from the SAR Imagery. This includes all the waterbodies present in the region
area_file = ['MATLAB_Processed_NE_JJAS_stats_area.csv','MATLAB_Processed_NW_JJAS_stats_area.csv','MATLAB_Processed_SE_JJAS_stats_area.csv','MATLAB_Processed_SW_JJAS_stats_area.csv']

for region in range(len(regions)):
    f = pd.read_csv(r'Path to directory where the A-V Files are saved//'+f_files[region])
    f['constant'] = 1
    f=f.rename(columns = {"esm_mean":"Mean_Vol(km3)",'esm_min':'Min_Vol(km3)','esm_max':'Max_Vol(km3)'})
    gauges = list(set(f['Name']))
    f['Date'] = pd.to_datetime(f['Date'], format='%Y-%m-%d')
    stats_area = pd.read_csv(r'Path to directory where the MATLAB Processed Area files are saved//'+area_file[region])
    area_greater_than_30 = stats_area['Area (inkm2, value*10*10/(1000*1000))'].values > 30

    
    # Run the function in parallel for each subset size from 1 to 30 for twenty thousand iterations.
    start_time = time.time()
    batch_size = 1000  # Define the size of your batch
    num_batches = 20000 // batch_size
    for batch in range(num_batches):  # twenty thousand iterations split into batches
        results = []
        for _ in range(batch_size):
            for i in range(1, len(gauges) + 1):
                selected_indices = np.random.choice(gauges, i, replace=False)
    #             print(gauges[i])
                result = compute(selected_indices,regions[region])
                results.append(result)
    #     # Save the results after each batch
        results_df = pd.DataFrame(list(results), columns=['Gauge_Combination','Slope','Cummulative_Volume'])
        results_df['Gauge_Combination'] = results_df['Gauge_Combination'].apply(lambda x: np.array2string(x, separator=','))
        results_df.to_csv(rf'Path to Directory where Monte Carlo Simulations will be saved//'+regions[region]+'/ML/ML_Random_Monte-Carlo_'+str(batch+1)+'.csv', index=False)

    end_time = time.time()
    elapsed_time = (end_time - start_time)/60

    print("Elapsed Time:", elapsed_time, "minutes")